<a href="https://colab.research.google.com/github/msatkun/MSc_Project/blob/main/Intel_neural_compressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Intel's Neural Compressor
https://github.com/intel/neural-compressor

In [1]:
!pip install neural-compressor
!pip install spacy
!pip install "portalocker[redis]"
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.1/241.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.7 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

# Load the IMDb dataset from hugging face
imdb_dataset = load_dataset("imdb")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Reference used:
https://colab.research.google.com/github/pytorch/text/blob/master/examples/legacy_tutorial/migration_tutorial.ipynb

https://github.com/intel/neural-compressor/blob/master/examples/tensorflow/nlp/transformer_lt/quantization/ptq/README.md

In [8]:
class Dataset(object):
  def __init__(self, dataset):
    # Initialize dataset related info here
    self.batch = dataset["text"]
    self.ref_lines = dataset["label"]

  def __getitem__(self, index):
    data = self.batch[index]
    label = self.ref_lines[index]
    return data, label

  def __len__(self):
    return len(self.batch)

In [9]:
imdb_data = Dataset(imdb_dataset["train"])

In [11]:
from neural_compressor import quantization
from neural_compressor.data import DataLoader
from neural_compressor.config import PostTrainingQuantConfig

ds = Dataset(FLAGS.inputs_file, FLAGS.reference_file, FLAGS.vocab_file)
calib_dataloader = DataLoader(dataset=ds, collate_fn=collate_fn,
                              batch_size=FLAGS.batch_size, framework='tensorflow')
conf = PostTrainingQuantConfig(inputs=['input_tensor'],
                               outputs=['model/Transformer/strided_slice_19'],
                               calibration_sampling_size=[500])
q_model = quantization.fit(graph, conf=conf, calib_dataloader=calib_dataloader,
                           eval_func=eval_func)
try:
  q_model.save(FLAGS.output_model)
except Exception as e:
  print("Failed to save model due to {}".format(str(e)))

AttributeError: ignored